<a href="https://colab.research.google.com/github/tarabelo/PIAC-2526/blob/main/06_B%C3%BAsqueda_adaptativa_de_Grover_(GAS).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install qiskit[visualization]

# **Búsqueda adaptativa de Grover (GAS)**

Presentado en 1966 (https://arxiv.org/abs/quant-ph/9607014) busca encontrar el mínimo de una función $g(x)$ usando el algoritmo de búsqueda de Grover.

El proceso es como sigue:

1. Se elige un valor $x_0$ aleatorio y se computa $g(x_0)$
1. Se define un oráculo $f:\{0,1\}^n \rightarrow \{0,1\}$ de la forma:
$$
f(y) =
\begin{cases}
1, \text{si } g(y) < g(x_0)\\
0, \text{si } \text{ en otro caso}
\end{cases}
$$
1. Se usa el algoritmo de Grover para encontrar un $y$ ($\mathcal{O}\left(\sqrt{2^n}\right)$ evaluaciones).
1. Se hace $x_0=y$ y se vuelve al punto 2 hasta encontrar el mínimo.

# **Implementación del oráculo**

El oráculo GAS puede ser complejo de implementar.

Para una función QUBO es más simple.


### **Ejemplo**: Implementación de la función $g(x)=-2x_0x_2-x_1x_2-x_0+2x_1-3x_2$

Necesitamos 3 cúbits para representar las $x$.

Es fácil ver que $-7 \le g(x) \le 2$, por lo que necesitamos un mínimo de 4 bits para representar las $g(x).

Un ejemplo de circuito:

<br>
<center><img src="https://drive.google.com/uc?export=view&id=1IZPRKNSCJH4xcuNedx6iJMB8VU--OaOW" alt="Ejemplo oraculo GAS" width="700"  /></center>
<br>

Necesitamos implementar las puertas de suma.


# **Suma en el dominio de Fourier (_phase encoding_)**

Para hacer la suma, podemos codificar los coeficientes mediante codificación en la fase.

Así, un número entero $j$ de $n$-bits tiene la siguiente codificación:

$$
\text{QFT}(|j\rangle) = \frac{1}{\sqrt{2^n}} \sum_{k=0}^{2^n-1} \exp\left(\frac{2\pi i}{2^n}jk\right)|k\rangle
$$

Por ejemplo, el 0 se representa como:

$$
\frac{1}{\sqrt{2^n}} \sum_{j=0}^{2^n-1} \exp\left(\frac{2\pi i}{2^n}0j\right)|j\rangle = \frac{1}{\sqrt{2^n}} \sum_{j=0}^{2^n-1} |j\rangle
$$

Ya que la QFT de $|0\rangle$ es $|+\rangle^{\otimes n}$


Sea un entero $j \ge 0$ codificado en la fase al que queremos sumarle otro entero no negativo $l$ para obtener:

$$
\frac{1}{\sqrt{2^n}} \sum_{k=0}^{2^n-1} \exp\left(\frac{2\pi i}{2^n}(j+l)k\right)|k\rangle
$$

Esta suma se puede hacer con puertas fase como se muestra en el siguiente circuito:

<center><img src="https://drive.google.com/uc?export=view&id=1ZSkZZhdM9KLNMvITQbro4ic9yYxcPuPU" alt="Suma phase encoding" width="300"  /></center>

Es decir, al cúbit $k_h$ se le aplica la puerta

$$
P\!\left(\frac{\pi}{2^{n-1-h}}\,l\right)
$$



## Funcionamiento
Sea $|k\rangle = |k_{n-1}\ldots k_0\rangle$ un estado base. Sea $k_h\in\{0,1\}$ el bit $h$-ésimo:

- Si el qubit $k_h$ vale 0 → la puerta no introduce fase.  
- Si el qubit $k_h$ vale 1 → añade fase  $\exp\left(\frac{i\pi\,l}{2^{n-1-h}}\right)$.


El estado $|k\rangle$ recibe por tanto la fase total:

$$
\prod_{h=0}^{n-1}
\exp\left(\frac{i \pi l\, k_h}{2^{n-1-h}}\right)
=
\exp\left({i\pi l \sum_{h=0}^{n-1} \frac{k_h}{2^{n-1-h}}}\right).
$$

Usando la expansión binaria, el entero $k$ se puede escribir:

$$
k = \sum_{h=0}^{n-1} k_h 2^h \implies \frac{k}{2^{n-1}} = \sum_{h=0}^{n-1} \frac{k_h}{2^{n-1-h}}
$$


Por tanto, la fase total aplicada resulta:

$$
\exp\left(\frac{i \pi l k}{2^{n-1}}\right)
= \exp\left(\frac{2i \pi l k}{2^{n}}\right).
$$

## Resultado final
Aplicar este circuito a un entero $j$ codificado en la fase como

$$
\frac{1}{\sqrt{2^n}}
\sum_{k=0}^{2^{n-1}} \exp\left(\frac{2\pi i}{2^n}jk\right)|k\rangle
$$

produce:

$$
\boxed{
\frac{1}{\sqrt{2^n}}
\sum_{k=0}^{2^{n-1}} \exp\left(\frac{2\pi i}{2^n}(j+l)k\right)|k\rangle
}
$$

lo cual implementa la suma de enteros en codificación de fase.

Aplicando la QFT inversa:

$$
\text{QFT}^{-1}\left(\frac{1}{\sqrt{2^n}}
\sum_{k=0}^{2^{n-1}} \exp\left(\frac{2\pi i}{2^n}(j+l)k\right)|k\rangle\right) = j+l
$$

### Números negativos

Usando complemento a 2 para representar enteros negativos, la representación binaria de $l\lt 0$ es:

$$
l = 2^n-|l| = 2^n + l
$$

Por lo tanto, la fase aplicada por el circuito anterior al $h$-ésimo bit será:

$$
\exp\left(\frac{i\pi(2^n+l)}{2^{n-1-h}}\right) = \exp\left(\frac{i\pi\,2^n}{2^{n-1-h}}\right) \exp\left(\frac{i\pi\,l}{2^{n-1-h}}\right) = \exp\left(\frac{i\pi\,l}{2^{n-1-h}}\right).
$$

ya que:

$$
\exp\left(\frac{i\pi\,2^n}{2^{n-1-h}}\right) = \exp\left(i\pi\,2^{h+1}\right) = 1
$$

Es decir, el circuito nos sirve para números negativos y positivos.

**Ejemplo**: Función que devuelve una puerta que suma un entero $l$ en phase encoding

In [ ]:
from qiskit import QuantumCircuit, QuantumRegister
from qiskit.quantum_info import Statevector
from qiskit.circuit import Gate
import numpy as np

def pAddGate(l: int, n: int) -> Gate:
  """
  Devuelve un qiskit.circuit.Gate que ejecuta una suma +l en el dominio de Fourier
  de un valor entero de n cúbits.
  n debe ser suficientemente grande para representar el valor de la suma

  Argumentos:
    l: valor a sumar (int)
    n: numero de cúbits (int)
  """
  qc = QuantumCircuit(n)

  for h in range(n):
    qc.p((np.pi*l)/2**(n-1-h),h)

  gate = qc.to_gate(label="Suma "+str(l))

  return(gate)

Sumamos dos enteros a+b de 4 bits en complemento a 2 $\implies$ $-8 \le a,b \le 7$ y $-8 \le a+b \le 7$



In [ ]:
n = 4

a = 5
b = -7

qc = QuantumCircuit(n)

s1 = pAddGate(a,n)
s2 = pAddGate(-7,n)

# 0 en el espacio de fases
qc.h(range(n))

qc.append(s1, list(range(n)))
qc.append(s2, list(range(n)))

qc.draw('mpl')

In [ ]:
# Vemos el vector de estado
e = Statevector.from_instruction(qc)
e.draw('latex')

In [ ]:
# Aplicamos la QFT inversa
from qiskit.circuit.library import QFTGate

qc_inverso = qc.compose(QFTGate(n).inverse())

e = Statevector.from_instruction(qc_inverso)
e.draw('latex')

### **Ejemplo**: Implementación de la función $g(x)=-2x_0x_2-x_1x_2-x_0+2x_1-3x_2$ (continuación)


Tenemos que convertir la puerta de suma en una puerta controlada.

In [ ]:
from qiskit.circuit.controlledgate import ControlledGate

def CpAddGate(l: int, n: int, num_ctrl_qubits=1) -> ControlledGate:
  """
  Devuelve un qiskit.circuit.controlledgate.ControlledGate
  que ejecuta una suma +l en el dominio de Fourier
  de un valor entero de n cúbits controlada por
  num_ctrl_qubits.
  n debe ser sufucientemente grande para reprersentar el valor de la suma

  Argumentos:
    l: valor a sumar (int)
    n: numero de cúbits (int)
    num_ctrl_qubits: Numero de bits de control
  """
  qc = QuantumCircuit(n)

  # Le añado una Gate suma l no controlada
  ncgate = pAddGate(l, n)
  qc.append(ncgate, list(range(n)))

  # Convierto el circuito en una puerta controlada
  # con num_ctrl_qubits cúbits de control
  gate = qc.to_gate(label=ncgate.label)
  gate = gate.control(num_ctrl_qubits)

  return(gate)

Creo ahora un circuito que devuelva $g(x)$

In [ ]:
def gcircuit(n: int, nancillas: int, inverse = False) -> QuantumCircuit:
  """
  Creo el circuito para g(x)=-2x_0x_2 -x_1x_2 -x_0 + 2x_1 - 3x_2
  n = nº de variables
  nancillas: ancillas en las que se guarda el resultado
  El resultado se devuelve en base Fourier
  El valor g(x) se devuelve en las ancillas

  inverse: si True devuelve el circuito inverso para la descomputación
  """

  x = QuantumRegister(n, name='x')
  ancillas = QuantumRegister(nancillas, name='ancillas')

  qc = QuantumCircuit(x,ancillas)

  # Lista con los coeficientes y las variables de la función
  parameters = [([x[0],x[2]] , -2),
                ([x[1],x[2]] , -1),
                ([x[0]] , -1),
                ([x[1]] , 2),
                ([x[2]] , -3)]

  # Creo las puertas controladas
  if not inverse:
    puertas = list()
    for p in parameters:
      puertas.append(CpAddGate(p[1], nancillas, len(p[0])))

    # Pongo las ancillas a 0 en el dominio de Fourier
    qc.h(ancillas)

    # Añado las puertas controladas al circuito
    # Los cúbits de control hay que indicarlos como una lista
    # primero los qubits de control, luego los target
    for g in range(len(puertas)):
      qc.append(puertas[g], parameters[g][0]+list(ancillas))

  # Circuito inverso
  else:
    puertas = list()
    # Defino las puertas con el signo cambiado
    for p in parameters:
      puertas.append(CpAddGate(-p[1], nancillas, len(p[0])))
    # Añado las puertas controladas al circuito
    # Los cúbits de control hay que indicarlos como una lista
    # primero los qubits de control, luego los target
    for g in range(len(puertas)-1,-1,-1):
      qc.append(puertas[g], parameters[g][0]+list(ancillas))
    # Añado las Hadamard
    qc.h(ancillas)

  # Devuelvo el circuito
  return(qc)

In [ ]:
puertas=[1,2,3,4,5,6]
for i in range(len(puertas)-1,-1,-1):
  print(i)

In [ ]:
from qiskit.quantum_info import Statevector
from qiskit.circuit.library import QFTGate

n = 3
nancillas = 4

qc = gcircuit(n, nancillas)

# Uso la transformada inversa de Fourier para obtener el resultado
qc.append(QFTGate(nancillas).inverse(), list(range(n,n+nancillas)))

qc.draw('mpl')

In [ ]:
# Calculamos los diferentes valores de g
for i in range(2**n):
  estado_inicial = Statevector.from_int(i, 2**(n+nancillas))

  estado_final = estado_inicial.evolve(qc)
  display(estado_final.draw('latex'))

### Creación del oráculo

A partir de este circuito, podemos crear el oráculo que devuelva 1 cuando $g(y) \lt g(x_0)$, o, lo que es lo mismo, $g(y)-g(x_0) \lt 0$.

Suponemos $x_0$ y $g(x_0)$ conocidos.

Empezamos implementando el circuito que devuelva $g(y)-g(x_0)$ para  $g(x)=-2x_0x_2-x_1x_2-x_0+2x_1-3x_2$ (continuación)


In [ ]:
def restag(n:int, nancillas:int, init_value:int, inverse = False) -> QuantumCircuit:
  """
  Si inverse = False, devuelve el circuito que calcula g(y)-g(x_0)
  El circuito devuelve el resultado en base Z (aplica QFT inversa).

  Si inverse = True, devuelve el circuito inverso para la descomputación

  Parameters:
    n: nº de variables
    nancillas: ancillas en las que se guarda el resultado
    init_value: valor de g(x_0)
  """

  if not inverse:
    # Circuito que implementa g(y)
    qc = gcircuit(n, nancillas)

    # Puerta resta g(x_0)
    qc.append(pAddGate(-init_value, nancillas), list(range(n,n+nancillas)))

    # Uso la transformada inversa de Fourier para obtener el resultado
    qc.append(QFTGate(nancillas).inverse(), list(range(n,n+nancillas)))

  else:
    qc = QuantumCircuit(n+nancillas)
    # Añado la QFT directa
    qc.append(QFTGate(nancillas), list(range(n,n+nancillas)))

    # Puerta suma g(x_0)
    qc.append(pAddGate(init_value, nancillas), list(range(n,n+nancillas)))

    qc.compose(gcircuit(n, nancillas, inverse=True), inplace=True)

  return(qc)

In [ ]:
def oraculo(n:int, nancillas:int, init_value:int) -> Gate:
  """
  Devuelve el oráculo completo, como un circuito con n+nancillas+1 cúbit
  Esta ancilla adicional es la salida del oráculo
  """

  qc = QuantumCircuit(n+nancillas+1)

  qc.compose(restag(n,nancillas,init_value), inplace=True)

  # La última de las ancillas es 1 en el caso de g(y)-g(x_0) < 0
  qc.cx(n+nancillas-1, n+nancillas)

  # Uncomputation
  qc.compose(restag(n, nancillas, init_value, inverse=True), inplace=True)


  return(qc.to_gate(label='Oraculo GAS'))


In [ ]:
n = 3
nancillas = 4
# Tomo x_0 = 1
x = [1,0,0]
init_value = -2*x[0]*x[2]-x[1]*x[2]-x[0]+2*x[1]-3*x[2]

gate_oraculo = oraculo(n, nancillas, init_value)

x = QuantumRegister(n, name='x')
ancillas = QuantumRegister(nancillas, name='ancillas')
output =  QuantumRegister(1, name='out')

qc_oraculo = QuantumCircuit(x, ancillas, output)

qc_oraculo.compose(gate_oraculo, inplace=True)

qc_oraculo.draw('mpl')

In [ ]:
qc_oraculo.decompose().draw('mpl')